In [1]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

--2024-08-26 14:54:15--  https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [image/jpeg]
Saving to: ‘La_noche_estrellada1.jpg’

La_noche_estrellada 100%[===================>] 218.48K  --.-KB/s    in 0.1s    

2024-08-26 14:54:16 (1.84 MB/s) - ‘La_noche_estrellada1.jpg’ saved [223725/223725]

--2024-08-26 14:54:16--  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153015 (149K) [ima

In [2]:
!pip uninstall tensorflow -y
!pip install tensorflow==2.8.0

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully

In [3]:
#import os
#os.environ["KERAS_BACKEND"] = "jax"
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array
from tensorflow.keras import layers
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from tensorflow.keras.applications import vgg19
from pathlib import Path

In [22]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("/content/775px-Neckarfront_Tübingen_Mai_2017.jpg")
style_reference_image_path = Path("/content/La_noche_estrellada1.jpg")
result_prefix = Path("/content/output")
iterations = 100

## (1) ¿Qué significan los parámetros definidos en la siguiente celda?

In [23]:
total_variation_weight = 0.1 # peso de la variation loss, regulariza, típico en generación de imágenes
style_weight = 10 # peso del estilo, beta
content_weight = 1 # peso del contenido, alpha

In [24]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

## (2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

```python
img = load_img(image_path, target_size=(img_nrows, img_ncols))
```
Se carga la imagen desde el path especificado, escalándola al tamaño indicado por `target_size`. [La imagen se carga en formato PIL](https://www.tensorflow.org/api_docs/python/tf/keras/utils/load_img). \
 \
```python
    img = img_to_array(img)
```
Se convierte la imagen a un array de numpy, tridimensional, con shape `(height, width, channels)`: ya que no se provee el argumento `data_format`, los canales ocupan el último lugar del array (ver [código fuente](https://www.tensorflow.org/api_docs/python/tf/keras/utils/img_to_array)). \
 \
```python
    img = np.expand_dims(img, axis=0)
```
Se agrega una dimensión al array en el índice cero. Esto se debe a que el modelo [utiliza capas Conv2D](https://github.com/keras-team/keras/blob/fa834a767bfab5d8e4180ada03fd0b7a597d6d55/keras/src/applications/vgg19.py#L129-L189) de Keras que necesitan una [entrada con shape `(batch_size, height, width, channels)`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D). \
 \
```python
    img = vgg19.preprocess_input(img)
```
Los modelos de Keras Application esperan que las entradas sean preprocesadas de una manera particular, definida para cada modelo. En el caso de VGG19, la transformación realizada es [convertir las imágenes de RGB a BGR y eliminar la media respecto al dataset de ImageNet en cada canl](https://keras.io/api/applications/vgg/#vgg16-function). Esto se implementó en general para todas las Keras Applications mediante una única función `_preprocess_numpy_input` definida en `imagenet_utils` que en el caso de VGG19 es llamada con el argumento `caffe`. En [la definición](https://github.com/keras-team/keras/blob/fa834a767bfab5d8e4180ada03fd0b7a597d6d55/keras/src/applications/imagenet_utils.py#L161-L231) se verifica el comportamiento, es notable que las medias de ImageNet estén hardcodeadas en el código. También es notable que pareciera que esta función se puede llamar antes o después del `expand_dims`, en base a lo que veo de la implementación.

In [25]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

## (3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Hace la operación inversa a la celda anterior, recupera la información de la imagen en el formato original:

- Recupera la shape original
- Restaura las medias eliminadas en el preprocess de VGG19 (del source code de `imagenet_utils`, `mean = [103.939, 116.779, 123.68]`, afortunadamente coincide con lo que se puso acá)
- Pasa el orden de los canales de BGR a RGB,
- Castea a `uint8` (con "saturación", mediante el `clip`) ya que en el medio se realizaron operaciones con floats (como sumar las medias, en esta celda).

In [26]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [9]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [10]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

In [11]:

# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [12]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

model.summary()

80150528/80134624 [==============================] - 0s 0us/step
Model loaded.
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(3, 400, 517, 3)]        0         
                                                                 
 block1_conv1 (Conv2D)       (3, 400, 517, 64)         1792      
                                                                 
 block1_conv2 (Conv2D)       (3, 400, 517, 64)         36928     
                                                                 
 block1_pool (MaxPooling2D)  (3, 200, 258, 64)         0         
                                                                 
 block2_conv1 (Conv2D)       (3, 200, 258, 128)        73856     
                                                                 
 block2_conv2 (Conv2D)       (3, 200, 258, 128)        147584    
                                                

## (4) En la siguientes celdas:

- ¿Qué es la matriz de Gram? ¿Para qué se usa?
- ¿Por qué se permutan las dimensiones de x?

La matriz de Gram es una matriz construída a partir de los productos punto de los feature maps correspondientes a una determinada capa de una CNN. De cierta manera, esto permite obtener una medida de la correlación entre los filtros de la capa. En el contexto de style transfer, la matriz de Gram se usa para capturar información sobre la textura de cierta imagen, ya que en vez de analizar la configuración espacial de las features se trabaja con la correlación. Lo que se hará luego es generar una imagen cuya matriz de Gram tenga la menor distancia posible (MS) a la matriz de Gram de la imagen de referencia de estilo/textura.

Razonamiento: el argumento que se le pasa a esta función, que calcula la matriz de Gram, es un tensor de shape `(height, width, filters)`. La permutación deja un tensor de shape `(filters, height, width)` Al hacer `batch_flatten` se aplanan todas las dimensiones excepto por la primera, de manera que `features` tiene shape `(filters, height*width)`. De esta manera, al hacer la multiplicación mediante `dot`, la matriz resultante es de dimension `(filters, filters)`, que es lo que permite tener la correlación entre los distintos filtros de la capa, que es lo que queríamos. La permutación permite que el resultado esté dado como el producto de las componentes del feature map de la manera que especifica el paper:

$$G_{ij}^l=\sum_k F_{ik}^l F_{jk}^l$$

In [13]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

## (5) Losses

Explicar qué mide cada una de las losses en las siguientes tres celdas.

- Style loss: mide la distancia (mean-squared) entre la matriz de Gram de la imagen siendo generada y la de la imagen de referencia. Es decir, mide qué tan distintos son los estilos de la imagen generada y la de referencia.

- Content loss: mide la distancia entre las features de la imagen original y las de la imagen generada, es decir, mide qué tan distinta es la información contenida en ambas.

- Variation loss: viendo lo que hace la función particularmente, veo que penaliza el hecho de que dos pixeles contiguos sean muy distintos, ya que se hace mayor cuanto mayor es la diferencia que haya, tanto en x como en y, a una distancia de 1 pixel. Por lo tanto, mide qué tan "discontínua" es la imagen pixel a pixel, *infiero* que para penalizar cambios bruscos en los valores de pixeles y suavizar el resultado final.

In [14]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [15]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [16]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [17]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [18]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

## (6) Explique el propósito de las siguientes tres celdas

¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

In [19]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.


In [20]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

## (7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [21]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 12410916000.0
Image saved as /content/output/output_at_iteration_0.png
Iteration 0 completed in 476s
Start of iteration 1
Current loss value: 6141263000.0
Image saved as /content/output/output_at_iteration_1.png
Iteration 1 completed in 451s
Start of iteration 2
Current loss value: 4336484000.0
Image saved as /content/output/output_at_iteration_2.png
Iteration 2 completed in 458s
Start of iteration 3
Current loss value: 3195453200.0
Image saved as /content/output/output_at_iteration_3.png
Iteration 3 completed in 457s
Start of iteration 4
Current loss value: 2562114300.0
Image saved as /content/output/output_at_iteration_4.png
Iteration 4 completed in 456s
Start of iteration 5
Current loss value: 2263019800.0
Image saved as /content/output/output_at_iteration_5.png
Iteration 5 completed in 452s
Start of iteration 6


KeyboardInterrupt: 

## (8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)


## (9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.
